In [1]:
import os
import glob
import pandas as pd
import numpy as np
from multiprocessing import Pool

### sQTLSeekeR

In [8]:
file = "/home/galaxy/data/QTL/sQTL/sQTLSeekeR_Transcript-Ratio/sQTLSeekeR_Transcript-Ratio.csv"
common_vcf = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GRCh38_common_all_20180418.vcf"
result_dir = "/home/galaxy/data/QTL/sQTL/sQTLSeekeR_Transcript-Ratio/info_dir"

In [6]:
df_vcf = pd.read_table(common_vcf, sep="\t", header=None, comment="#")
df_vcf.columns = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]
df_vcf_sub = df_vcf[["CHROM", "POS", "ID"]]

/home/galaxy/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
df = pd.read_csv(file)
df_sub = df[["Gencode Id", "Gene Symbol", "Tissue", "SNP", "Event", "Transcript 1", "Transcript 2"]]
# column "SNP" contains blank space at the end.
df_sub["ID"] = [x.strip() for x in df["SNP"]]
df_sub.head()

/home/galaxy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Gencode Id,Gene Symbol,Tissue,SNP,Event,Transcript 1,Transcript 2,ID
0,ENSG00000001460.12,C1orf201,Adipose_Subcutaneous,rs1016209,exon skipping,ENST00000337248.4,ENST00000003583.8,rs1016209
1,ENSG00000001460.12,C1orf201,Adipose_Subcutaneous,rs11589294,exon skipping,ENST00000337248.4,ENST00000003583.8,rs11589294
2,ENSG00000001460.12,C1orf201,Adipose_Subcutaneous,rs11587667,exon skipping,ENST00000337248.4,ENST00000003583.8,rs11587667
3,ENSG00000001460.12,C1orf201,Adipose_Subcutaneous,rs7529526,exon skipping,ENST00000337248.4,ENST00000003583.8,rs7529526
4,ENSG00000001460.12,C1orf201,Adipose_Subcutaneous,rs12140084,complex event 5';complex event;complex event 3',ENST00000374404.2,ENST00000440416.1,rs12140084


In [16]:
tissue_list = list(set(df_sub.Tissue.tolist()))
for tissue in tissue_list:
    print(tissue)
    df_i = df_sub[df_sub.Tissue == tissue]
    df_merge = df_i.merge(df_vcf_sub, how="left", on="ID").dropna().drop_duplicates()
    result_file = os.path.join(result_dir, "%s.txt" % tissue)
    df_merge.to_csv(result_file, sep="\t", index=False)

Nerve_Tibial
Skin_Sun_Exposed_Lower_leg
Thyroid
Lung
Heart_Left_Ventricle
Whole_Blood
Adipose_Subcutaneous
Artery_Aorta
Muscle_Skeletal


#### GENERATE BED FILES

In [21]:
data_dir = "/home/galaxy/data/QTL/sQTL/sQTLSeekeR_Transcript-Ratio/info_dir"
result_dir = "/home/galaxy/data/QTL/sQTL/sQTLSeekeR_Transcript-Ratio/GRCh38_bed"
file_list = glob.glob("%s/*.txt" % data_dir)
for x in file_list:
    df = pd.read_table(x, sep="\t")
    result_file = os.path.join(result_dir, os.path.basename(x).replace(".txt", ".bed"))
    df["info"] = df["Gencode Id"] + ";" + df["Gene Symbol"] + ";" + df["Tissue"] + ";" + df["Event"] + ";" + df["Transcript 1"] + ";" + df["Transcript 2"] + ";" + df["ID"] 
    df["chr_name"] = "chr" + df["CHROM"].astype(str)
    df["start"] = df["POS"].astype(int) - 1
    df["end"] = df["POS"].astype(int)
    df_final = df[["chr_name", "start", "end", "info"]]
    df_final.to_csv(result_file, sep="\t", header=False, index=False)

### Altrans

In [23]:
data_dir = "/home/galaxy/data/QTL/sQTL/Altrans_Exon-Junction"
result_dir = "/home/galaxy/data/QTL/sQTL/Altrans_Exon-Junction/info_dir"

In [ ]:
df_vcf = pd.read_table(common_vcf, sep="\t", header=None, comment="#")
df_vcf.columns = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]
df_vcf_sub = df_vcf[["CHROM", "POS", "ID"]]

In [26]:
#########
file_list = glob.glob("%s/*.csv" % data_dir)
for x in file_list:
    print(x)
    df = pd.read_csv(x)
    result_file = os.path.join(result_dir, os.path.basename(x))
    df = df[df["SNP ID"].str.startswith("rs")]
    df_sub = df[["SNP ID", "Tissue", "Exon 1 Gencode Id", "Exon 2 Gencode Id"]]
    df_merge = df_sub.merge(df_vcf_sub, how="left", left_on="SNP ID", right_on="ID").dropna().drop_duplicates()
    result_file = os.path.join(result_dir, os.path.basename(x))
    df_merge.to_csv(result_file, sep="\t", index=False)

In [28]:
data_dir = "/home/galaxy/data/QTL/sQTL/Altrans_Exon-Junction/info_dir"
result_dir = "/home/galaxy/data/QTL/sQTL/Altrans_Exon-Junction/GRCh38_bed"
file_list = glob.glob("%s/*.csv" % data_dir)
for x in file_list:
    df = pd.read_table(x, sep="\t")
    result_file = os.path.join(result_dir, os.path.basename(x).replace(".csv", ".bed"))
    df["info"] = df["Tissue"] + ";" + df["Exon 1 Gencode Id"] + ";" + df["Exon 2 Gencode Id"] + ";" + df["SNP ID"]
    df["chr_name"] = "chr" + df["CHROM"].astype(str)
    df["start"] = df["POS"].astype(int) - 1
    df["end"] = df["POS"].astype(int)
    df_final = df[["chr_name", "start", "end", "info"]]
    df_final.to_csv(result_file, sep="\t", header=False, index=False)

#### nature communications 2016

In [12]:
############################################################################
raw_file = "/home/galaxy/data/QTL/sQTL/nc_2016_Brain/ncomms14519-s2.txt"
result_file = "/home/galaxy/data/QTL/sQTL/nc_2016_Brain/hg19_ncomms14519-s2.bed"
############################################################################
df = pd.read_table(raw_file, sep="\t")
df["chr_name"] = df["SNP_position(hg19)"].str.split(":").str[0]
df["start"] = df["SNP_position(hg19)"].str.split(":").str[1].astype(int) - 1
df["end"] = df["SNP_position(hg19)"].str.split(":").str[1].astype(int)
df["name"]=df["sQTL SNP_ID"]+";"+df["Ref"]+";"+df["Alt(all_observed_alleles)"]+";"+df["Function"]+";"+df["AS_position(hg19)c"]+";"+df["Gene"]
df_final = df[["chr_name", "start", "end", "name"]]
df_final.to_csv(result_file, sep="\t", header=False, index=False)
# df.head()

#### nature genetics 2015

In [13]:
#######################################################################
raw_file = "/home/galaxy/data/QTL/sQTL/ng_2015_blood/ng.3220-S2.csv"
result_file = "/home/galaxy/data/QTL/sQTL/ng_2015_blood/hg19_ng.3220-S2.bed"
#######################################################################
df = pd.read_csv(raw_file)
# df.head()
df["chr_name"] = "chr" + df["chromosome"]
df["start"] = df["position"].astype(int) - 1
df["end"] = df["position"].astype(int)
df["name"] = df["rs"]+";"+df["GeneSymbol"]+";"+df["effect"]+";"+df["noneffect"]+";"+df["distance"].astype(str)
df_final = df[["chr_name", "start", "end", "name"]]
df_final.to_csv(result_file, sep="\t", header=False, index=False)

#### hg19 to hg38

In [15]:
common_vcf = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GRCh38_common_all_20180418.vcf"
df_vcf = pd.read_table(common_vcf, sep="\t", header=None, comment="#")
df_vcf.columns = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]
df_vcf_sub = df_vcf[["CHROM", "POS", "ID"]]

/home/galaxy/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
#####################################################################################
common_vcf = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GRCh38_common_all_20180418.vcf"
file_1 = "/home/galaxy/data/QTL/sQTL/nc_2016_Brain/hg19_ncomms14519-s2.bed"
file_2 = "/home/galaxy/data/QTL/sQTL/ng_2015_blood/hg19_ng.3220-S2.bed"
res_1, res_2 = file_1.replace("hg19", "GRCh38"), file_2.replace("hg19", "GRCh38")
######################################################################################
    
def hg19_to_hg38(in_file, out_file):
    df = pd.read_table(in_file, sep="\t", header=None, names=["chr", "s", "e", "name"]).dropna()
    df["ID"] = df.iloc[:, -1].str.split(";").str[0]
    df = df.merge(df_vcf_sub, how="left", on="ID").dropna()
    #
    df["start"] = df["POS"].astype(int) - 1
    df["end"] = df["POS"].astype(int)
    df_final = df[["chr", "start", "end", "name"]]
    df_final.to_csv(out_file, sep="\t", header=False, index=False)
#   
hg19_to_hg38(file_1, res_1)
hg19_to_hg38(file_2, res_2)